# Timeseries

Pandas started out in the financial world, so it naturally has strong support for timeseries data.
We'll look at some pandas data types and methods for manipulating timeseries data.
Afterwords, we'll use [statsmodels' state space framework](http://www.statsmodels.org/stable/statespace.html) to model timeseries data.

In [ ]:
import numpy as np
import pandas as pd
import seaborn.apionly as sns
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

In [ ]:
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 6)
pd.options.display.max_rows = 10

## Datatypes

- `pd.Timestamp` (nanosecond resolution `datetime.datetime`)
- `pd.Timedelta` (nanosecond resolution `datetime.timedelta`)

Pandas provides highly-performant (mostly) drop-in replacements for `datetime.datetime` (`pd.Timestamp`) and `datetime.tiemedelta` (`pd.Timedelta`).
These have been tailored for efficient storage in NumPy arrays.
For the most part you'll be working with `DatetimeIndex`es or `TimedeltaIndex`es, or Series / DataFrames containing these.

The biggest limitation is that pandas stores `Timestamp`s at nanosecond resolution. Since they're backed by NumPy's 64-bit integer, the minimum and maximum values are

In [ ]:
pd.Timestamp.min, pd.Timestamp.max

If this is a problem, [there are workarounds](http://pandas.pydata.org/pandas-docs/stable/timeseries.html#representing-out-of-bounds-spans).

We'll go back to the BTS data set on flights.
This time I've provided the number of flights per hour for two airports in Chicago: Midway (MDW) and O'Hare (ORD). The data go back to January 1st, 2000.

In [ ]:
df = pd.read_csv("data/flights-ts.csv.gz", index_col=0, parse_dates=True)
df.head()

## Resampling

Resampling is similar to a groupby, but specialized for datetimes.
Instead of specifying a column of values to group by, you  specify a `rule`: the desired output frequency.
The original data is binned into each group created by your rule.

In [ ]:
resampler = df.resample("MS")  # MS=Month Start
resampler

There's an extensive list of frequency codes: http://pandas.pydata.org/pandas-docs/stable/timeseries.html#offset-aliases.

If you examine the raw data in `df`, you'll notice that it's not at a fixed frequency.
Hours where there weren't any flights just simply aren't present.
This isn't a problem though; resample is perfect for going from "ragged" timeseries data to fixed-frequency data.

Just like with `.groupby`, `.resample` returns a deferred object that hasn't really done any work yet.
It has methods for aggregation, transformation, and general function application.

In [ ]:
resampler.sum()

In [ ]:
resampler.sum().plot();

<div class="alert alert-success" data-title="Resample">
  <h1><i class="fa fa-tasks" aria-hidden="true"></i> Exercise: Resample</h1>
</div>
<p>Plot the standard deviation for the number of flights from `MDW` and `ORD` at a weekly frequency</p>

In [ ]:
# Your solution


In [ ]:
%load solutions/timeseries_resample.py

<div class="alert alert-success" data-title="Resample-Agg">
  <h1><i class="fa fa-tasks" aria-hidden="true"></i> Exercise: Resample-Agg</h1>
</div>
<p>Compute the the total number of flights (sum), mean, and median flights *per Quarter*.</p>

In [ ]:
%load solutions/timeseries_resample_agg.py

## Rolling, Expanding

Applying functions to windows, moving through your data.

These are very similar to groupby and resample. Let's get the daily number of flights with a `resample` quick.

In [ ]:
daily = df.resample('D').sum()
daily

Suppose you wanted a 30-day moving (or rolling) average.
This is possible with the `.rolling` method. Like `groupby` and `resample`, this object is just going to store the information to know what subset of data to operate on next; it doesn't actually do any work yet:

In [ ]:
daily.rolling(30, center=True)

The first argument is the window size.
Since `daily` is at daily frequency, 30 means a 30-day window.
`center=True` says to label each window with the middle-most point.
To actually do work, you call a method like `.mean`;

In [ ]:
fig, ax = plt.subplots()
daily.rolling(30).mean().rename(columns=lambda x: x + " (30D MA)").plot(ax=ax, alpha=.25,
                                                                        color=['C0', 'C1'])
daily.plot(ax=ax, alpha=.25, color=['C0', 'C1'], legend=False);

It's common to combine resampling and rolling.

In [ ]:
df.resample("D").sum().rolling(30).corr(pairwise=True).loc[:, 'ORD', 'MDW'].plot(
    title="O'Hare : Midway cross-correlation (30D MA)", figsize=(12, 4)
);

## Timezones

pandas can store an array of datetimes with a common timezone.
Right now the index for `df` is timezone naïve, but we can convert to a timezone with `tz_convert`:

In [ ]:
df.index.tzinfo  # None, timezone naïve

In [ ]:
df.index.tz_localize("US/Central")

Timezones, as usual, are annoying to deal with.
We've hit a daylight savings time issue.
As the error says, 2000-04-02T02:00:00 isn't actaully a valid time in US/Central.
I checked the BTS website, and these timestamps are supposed to be local time, so presumably some data was recorded incorrectly.
pandas is strict by default, so it we need to tell it to ignore those errors: 

In [ ]:
idx = df.index.tz_localize("US/Central", ambiguous="NaT", errors='coerce')
idx

In [ ]:
pd.isnull(idx).sum()  # 25 bad values

Notice the dtype: `datetime64[ns, US/Central]`.
That means nanosecond resolution in the US/Central time zone.
Once you have a datetime with timezone, you can convert timezones with `tz_convert`:

In [ ]:
idx.tz_convert("UTC")

## Offsets

I wish the standard library `datetime` module had something like this.
Let's generate some fake data with `pd.date_range`

In [ ]:
dates = pd.date_range("2016-01-01", end="2016-12-31", freq='D')
dates

There are a whole bunch of offsets available in the `pd.tseries.offsets` namespace. For example, to move 3 business days into the future:

In [ ]:
dates + pd.tseries.offsets.BDay(3)

Or to move to the next month end:

In [ ]:
dates + pd.tseries.offsets.MonthEnd()

## Timedelta Math

Being able to add columns of dates and timedeltas turns out to be quite convenient.
Let's go all the way back to our first example with flight delays from New York airports.

In [ ]:
flights = pd.read_csv("data/ny-flights.csv.gz", parse_dates=['dep', 'arr'])
flights.head()

<div class="alert alert-success" data-title="Convert Timedelta">
  <h1><i class="fa fa-tasks" aria-hidden="true"></i> Exercise: Convert Timedelta</h1>
</div>
<p>Convert `flights.dep_delay` and `flights.arr_delay` to timedelta dtype.</p>

- Hint: recall our type conversion methods: `pd.to_*`
- Make new columns in `flights` called `dep_delay_td` and `arr_delay_td`
- Check the `unit` argument for the conversion method. The delay columns are in *minutes*.

In [ ]:
# Your solution


In [ ]:
%load solutions/timeseries_timedelta.py

<div class="alert alert-success" data-title="Timedelta Math">
  <h1><i class="fa fa-tasks" aria-hidden="true"></i> Exercise: Timedelta Math</h1>
</div>
<p>Compute the actual time the flight left, but adding the departure time `dep` and the delay `dep_delay`.

In [ ]:
%load solutions/timeseries_departure.py

# Modeling Timeseries

Timeseries are an interesting problem to model.
If we're lucky, we have a long history of past data that we can (maybe) use to predict the future.
We can exploit regularity in the timeseries (seasonal patterns, periods of high values are typically followed by another high value, etc.) to better predict the future.

Statsmodels has a nice framework for fitting timeseries models and evaluating their output.

In [ ]:
import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
import statsmodels.api as sm

Let's model Monthly flights from `ORD`.

In [ ]:
y = daily.ORD.resample("MS").sum()
y.plot();

That final value is odd because it's not a complete month. Let's drop it.

In [ ]:
y = daily.ORD.resample("MS").sum().iloc[:-1]
y.head()

It's common to estimate the parameters on *differenced* values.
That is, make a new series $y'$ where $y_t' = y_t - y_{t-1}$. Pandas makes this simple with the `.diff` method.

In [ ]:
y_prime = y.diff()
y_prime.head()

We'll drop that first NaN:

In [ ]:
y_prime = y.diff().dropna()
y_prime.plot();

Think back to regular linear regression: Predict some variable $y$ with some matrix $X$:

$y = \beta_0 + \beta_1 X_1 + \beta_2 X_2 ... + \beta_p X_p + \varepsilon$

When modelling timeseries, past values of $y$ make for good components of $X$.
We can do this with the pandas `.shift` method:

In [ ]:
y_prime.shift()

So the value for `2001-01-01` (-867) is now labeled `2000-02-01`. We can collect many of these with a list comprehension and a `concat`.

In [ ]:
lagged = pd.concat([y_prime.shift(i) for i in range(9)], axis=1,
                   keys=['y', 'L1', 'L2', 'L3', 'L4', 'L5', 'L6', 'L7', 'L8'])
lagged

In [ ]:
mod_lagged = smf.ols('y ~ L1 + L2 + L3 + L4 + L5 + L6 + L7 + L8', lagged)
res_lagged = mod_lagged.fit()

res_lagged.summary()

In [ ]:
ax = res_lagged.fittedvalues.plot(label="predicted", figsize=(12, 4), legend=True)
y_prime.plot(label="actual", legend=True);

In practice, you won't be doing the `shift`ing and `diff`ing yourself.
It's more convenient to let statsmodels do that for us.
Then we don't have to worry about un-differencing the fitted / predicted results to interpret them correctly.
Also, the solvers we'll see next are a bit more sophisticated than a linear regression.

## AutoRegressive Model

Predict $y_{t+1}$, given $y_0, y_1, \ldots y_t$

Let's fit an autoregressive (AR) model. Autoregressive part just means using past values of $y$ to predict the future (like we did above).
We'll use statsmodel's `SARIMAX` model. The AR part of SARIMAX is for autoregressive.
It also handles seasonality (**S**), differencing (**I** for integrated), moving average (**MA**), and exogenous regressors (**X**).

We'll stick to a simple AR(8) model (use the last 8 periods) with a single period of differencing.

In [ ]:
mod = smt.SARIMAX(y, order=(8, 1, 0))  # AR(8), first difference, no MA
res = mod.fit()

As usual with statsmodels, we get a nice summary with the fitted coefficeints and some test statistics (which we'll ignore)

In [ ]:
res.summary()

The results instance has all the usual attributes and methods, like `fittedvalues`.

In [ ]:
ax = res.fittedvalues.iloc[1:].plot(label="Fitted", legend=True, figsize=(12, 4))
y.plot(ax=ax, label="Actual", legend=True);

## Forecasting

The real value of timeseries analysis is to predict the future.
We can use the `.get_prediction` method to get the predicted values, along with a confidence interval.

First, we'll look at one-period-ahead forecasts.
Basically, this simulates looking at our data the last day of the month, and making the forecast for the next month.
Keep in mind though that we fit our parameters on the entire dataset. The isn't an out-of-sample prediction.

In [ ]:
pred = res.get_prediction(start='2001-03-01')
pred_ci = pred.conf_int()

In [ ]:
ax = y.plot(label='observed')
pred.predicted_mean.plot(ax=ax, label='Forecast', alpha=.7)
ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=.2)
plt.legend()
sns.despine()

Alternatively, we can make dynamic forecasts as of some month (January 2013 in the example below). That means the forecast from that point forward only use information available as of January 2013 (though again, we fit the model on the entire dataset). The predictions are generated in a similar way: a bunch of one-step forecasts. Only instead of plugging in the actual values beyond January 2013, we plug in the forecast values.

In [ ]:
pred_dy = res.get_prediction(start='2002-03-01', dynamic='2013-01-01')
pred_dy_ci = pred_dy.conf_int()

In [ ]:
ax = y.plot(label='observed')
pred_dy.predicted_mean.plot(ax=ax, label='Forecast')
ax.fill_between(pred_dy_ci.index,
                pred_dy_ci.iloc[:, 0],
                pred_dy_ci.iloc[:, 1], color='k', alpha=.25)
ylim = ax.get_ylim()
ax.fill_betweenx(ylim, pd.Timestamp('2013-01-01'), y.index[-1],
                 alpha=.1, zorder=-1)
ax.set_ylim(ylim)
ax.annotate('Dynamic $\\longrightarrow$', (pd.Timestamp('2013-02-01'), 15000))

plt.legend()
sns.despine()

There are *a lot* of issues we didn't cover here.
Seasonality, non-stationarity, autocorrellation, unit roots, and more.
Timeseries modeling is fraught with traps that will throw off your predictions.
Still, this should give you a taste of what's possbile.

## Further Resources

- [statsmodels state space docs](http://www.statsmodels.org/dev/statespace.html)
- [statsmodels state space examples](http://www.statsmodels.org/dev/examples/index.html#statespace)
- [pyflux](http://www.pyflux.com), another time series modeling library
- Sean Abu's [post on ARIMA](http://www.seanabu.com/2016/03/22/time-series-seasonal-ARIMA-model-in-python/)
- Jeffrey Yau's [talks at PyData](https://www.youtube.com/watch?v=tJ-O3hk1vRw)
- My [blog post](http://tomaugspurger.github.io/modern-7-timeseries.html)